In [2]:
%pip install langchain_huggingface langchain_core python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [25]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

prompt1 = template1.invoke({'topic':'black hole'})

result = model.invoke(prompt1)

prompt2 = template2.invoke({'text':result.content})

result1 = model.invoke(prompt2)

print(result1.content)


Here's a 5-line summary of the text:

- Black holes are incredibly dense areas in space with such strong gravity that even light cannot escape, making them "invisible".
-  They form when massive stars collapse, creating a singularity of infinite density surrounded by an "event horizon".
-  Black holes were theorized by Einstein and are divided into stellar-mass, supermassive, and intermediate-mass types based on their mass and influence on their surroundings.
- Although invisible, their effects are observable through things like gravitational lensing and superheated disks.
- Research into black holes focuses on understanding their formation, behavior, and potential connection to the fabric of spacetime. 



In [5]:
%pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 3.7 MB/s eta 0:00:00


In [28]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)
# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)

Black holes are regions of spacetime with such extreme gravity that nothing, not even light, can escape. Formed from the collapse of massive stars, black holes possess a singularity, an infinitely dense point, and an event horizon beyond which escape is impossible.  Supermassive black holes at galaxy centers play a crucial role in galaxy formation and evolution, while collisions of black holes produce powerful bursts of energy. Open questions surrounding black holes include the information paradox and the quantum nature of gravity. Research into black holes continues to unveil ever more mysteries while pushing the boundaries of our understanding of the universe. 



In [46]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} in JSON format {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)

{'facts': ['Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape.', 'They are formed when massive stars collapse at the end of their lives.', 'Black holes come in different sizes, with the most massive ones called supermassive black holes, found at the centers of galaxies.', "Despite their name, black holes aren't actually holes, but rather incredibly dense objects.", 'We can detect black holes indirectly through their gravitational influence on nearby stars and gas clouds.']}


In [47]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it", # Changed model to a chat-optimized one
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

parser = JsonOutputParser()

template = PromptTemplate(
    #template='Give me the name, age, and city of a fictional person in JSON format.', # Explicitly ask for JSON
    template = 'Give me 5 facts about {topic} in JSON format {format_instruction}',
    input_variables=['topic'],
    partial_variables= {'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser # The parser will handle the JSON string output

# prompt = template.format() # Removed this line

#print(prompt)

#result = chain.invoke({}) # Invoke with an empty dictionary as there are no input variables
result = chain.invoke({'topic':'Indian Education'})
# final_result = parser.parse(result.content) # Remove redundant parser call
print(result)

[{'fact': 'India boasts a diverse, long-standing education system.', 'detail': 'It combines ancient traditional learning with modern, global influences.  Vast variations exist across states and regions.'}, {'fact': 'Literacy rates have seen significant growth.', 'detail': 'Efforts throughout the nation have increased literacy, particularly among rural populations, but disparities between genders and socioeconomic backgrounds persist.'}, {'fact': 'English is a prevalent language of instruction, but language diversity is found throughout classroom settings.', 'detail': 'While English is widely taught at the higher levels, regional languages are often initially the starting point for child education'}, {'fact': 'India has a strong vocational and technical education sector.', 'detail': "At all levels, there's a focus on vocational skills development, especially in areas of STEM research, agriculture, and trade."}, {'fact': 'Substantial investment in public education is crucial for national

In [50]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'Pradhanmantri AWAS Yogna'})

print(result)

result2 = chain.invoke({'topic':'Indian Education after Graduation'})

print(result2)

{'fact_1': 'Pradhanmantri Awas Yojana (PMAY) is a flagship program of the Government of India launched in 2015.', 'fact_2': 'The scheme aims to provide affordable housing to all rural and urban residents across India, focusing on economically weaker sections and marginalized groups.', 'fact_3': 'PMAY provides funding for both the construction of new houses and also for renovating or improving existing ones.'}
{'fact_1': 'In India, there are a wide range of opportunities available after graduation, including government jobs, public sector organizations, NGOs, and private sector companies.', 'fact_2': "There's a significant focus on core skills development, practical knowledge and hands-on experience through internships and projects during the post degree work experience.", 'fact_3': 'The perceived value of higher education varies in different parts of the country, making career choices a thoughtful process based on individual priorities and the job market in various regions.'}


# Same with chain

In [51]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

prompt = template.format(topic='Pradhanmantri AWAS Yogna')

result = model.invoke(prompt)

result_final = parser.parse(result.content)

print(result_final)


{'fact_1': 'The Pradhan Mantri Awas Yojana (PMAY) is a flagship program aimed at providing affordable housing to all economically weaker sections of the society by 2022.', 'fact_2': 'It aims to achieve this goal through both financial assistance and support for land acquisition, construction, repair, and registration.', 'fact_3': 'The program is a major economic and social initiative of the Bharatiya Janata Party (BJP) government.'}


# Pydantic Output Parser

In [52]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

prompt = template.invoke({'place':'sri lankan'})

result = model.invoke(prompt)

final_result = parser.parse(result.content)
final_result

Person(name='Ranil Silva', age=32, city='Colombo')

In [53]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)
chain = template | model | parser

final_result = chain.invoke({'place':'Australia'})

print(final_result)

name='Zoe Reynolds' age=28 city='Melbourne'
